<a href="https://colab.research.google.com/github/MarrinXia/MSSP-607/blob/main/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Complete the following exercise:

a. Find the 95th percentile of earthquake magnitude in Japan using magType of 'mb'

b. Find the percentage of earthquakes in Indonesia that were coupled with tsunamis.

c. Get summary statistics for earthquakes in Nevada

d. Add a column to the dataframe indicating whether or not the earthquake happened in a country or US state that is on the Ring of Fire. Use Bolivia, Chile, Ecuador, Peru, Costa Rica, Guatemala, Mexico (be sure not to select New Mexico), Japan, Philippines, Indonesia, New Zealand, Antarctica (look for the Antarctic), Canada, Fiji, Alaska, Washington, California, Russia, Taiwan, Tonga, and the Kermadec Islands).

e. Calculate the number of earthquakes in the Ring of Fire locations and the number outside of them,

f. Find the tsunami count along the Ring of Fire.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re
import numpy as np
PATH = '/content/drive/MyDrive/parsed.csv'
df = pd.read_csv(PATH)
df.columns = [c.strip() for c in df.columns]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def pick_first_present(cands, cols):
    for c in cands:
        if c in cols:
            return c
    return None

cols = set(df.columns)
mag_col     = pick_first_present(["mag", "magnitude"], cols)
magtype_col = pick_first_present(["magType", "magtype", "mag_type"], cols)
tsunami_col = pick_first_present(["tsunami", "tsunami_flag", "tsunami_ind"], cols)

text_cols = [c for c in ["place","parsed_place","location","region","where","title","place_name"] if c in df.columns]
if text_cols:
    df["_text"] = (
        pd.Series([""]*len(df))
        .str.cat([df[c].astype(str) for c in text_cols], sep=" | ")
        .str.lower()
        .fillna("")
    )
else:
    df["_text"] = ""
if mag_col is not None:
    df[mag_col] = pd.to_numeric(df[mag_col], errors="coerce")

if tsunami_col is not None:
    tsu = pd.to_numeric(df[tsunami_col], errors="coerce")
    if tsu.isna().all():  # 有些数据用 "true/false/yes/no"
        tsu_txt = df[tsunami_col].astype(str).str.lower()
        tsu = tsu_txt.isin(["1","true","t","yes","y"]).astype(int)
    df[tsunami_col] = tsu.fillna(0).astype(int)


In [ ]:
#a
def japan_mb_p95():
    if (mag_col is None) or (magtype_col is None):
        return np.nan
    mask = df["_text"].str.contains(r"\bjapan\b", na=False) & (df[magtype_col].astype(str).str.lower() == "mb")
    vals = df.loc[mask, mag_col].dropna().values
    return float(np.percentile(vals, 95)) if len(vals) else np.nan

a_95 = japan_mb_p95()
print("(a) Japan + magType=mb → 95th percentile:", "N/A" if pd.isna(a_95) else round(a_95,3))


(a) Japan + magType=mb → 95th percentile: 4.9


In [ ]:
#b
def indonesia_tsunami_pct():
    if tsunami_col is None:
        return np.nan
    mask = df["_text"].str.contains(r"\bindonesia\b", na=False)
    sub = df.loc[mask]
    if sub.empty:
        return np.nan
    return 100.0 * (sub[tsunami_col] == 1).mean()

b_pct = indonesia_tsunami_pct()
print("(b) Indonesia tsunami %:", "N/A" if pd.isna(b_pct) else f"{b_pct:.2f}%")


(b) Indonesia tsunami %: 23.13%


In [ ]:
#c
def nevada_summary():
    mask = df["_text"].str.contains(r"\bnevada\b", na=False)
    sub = df.loc[mask].copy()
    if sub.empty:
        return pd.DataFrame({"note":["No Nevada rows found"]})
    # 尽力把可数值列转成数值
    for c in sub.columns:
        if not pd.api.types.is_numeric_dtype(sub[c]):
            sub[c] = pd.to_numeric(sub[c], errors="ignore")
    num_cols = sub.select_dtypes(include="number").columns
    return sub[num_cols].describe().T if len(num_cols) else pd.DataFrame({"note":["No numeric columns"]})

summary_nv = nevada_summary()
summary_nv


/tmp/ipython-input-4066945024.py:10: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  sub[c] = pd.to_numeric(sub[c], errors="ignore")


,count,mean,std,min,25%,50%,75%,max
alert,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cdi,15.0,2.440000e+00,5.011416e-01,2.000000e+00,2.000000e+00,2.200000e+00,2.900000e+00,3.300000e+00
code,681.0,9.270779e+05,3.686431e+06,6.570310e+05,6.580340e+05,6.590890e+05,6.598570e+05,7.309656e+07
dmin,681.0,1.661985e-01,1.662284e-01,1.000000e-03,5.300000e-02,1.120000e-01,2.330000e-01,1.414000e+00
felt,15.0,2.400000e+00,4.626013e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.900000e+01
gap,681.0,1.536681e+02,6.873530e+01,2.914000e+01,9.738000e+01,1.491400e+02,1.997200e+02,3.559100e+02
mag,681.0,5.000734e-01,6.967100e-01,-5.000000e-01,-1.000000e-01,4.000000e-01,9.000000e-01,2.900000e+00
mmi,1.0,2.840000e+00,NaN,2.840000e+00,2.840000e+00,2.840000e+00,2.840000e+00,2.840000e+00
nst,681.0,1.261821e+01,9.866963e+00,3.000000e+00,6.000000e+00,1.000000e+01,1.600000e+01,6.100000e+01
rms,681.0,1.519860e-01,8.466216e-02,5.000000e-04,1.069000e-01,1.463000e-01,1.871000e-01,8.634000e-01


In [ ]:
#d
ring_patterns = [
    r"\bbolivia\b", r"\bchile\b", r"\becuador\b", r"\bperu\b",
    r"\bcosta\s+rica\b", r"\bguatemala\b",
    r"\bmexico\b",  # 下面会先把 "new mexico" 从文本中删掉
    r"\bjapan\b", r"\bphilippines\b", r"\bindonesia\b", r"\bnew\s+zealand\b",
    r"\bantarctica\b", r"\bantarctic\b",
    r"\bcanada\b", r"\bfiji\b", r"\balaska\b", r"\bwashington\b", r"\bcalifornia\b",
    r"\brussia\b", r"\btaiwan\b", r"\btonga\b", r"\bkermadec\b", r"\bkermadec\s+islands\b"
]
ring_regexes = [re.compile(p) for p in ring_patterns]

def on_ring_of_fire(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.lower()
    t = t.replace("new mexico", "")  # 先去掉“new mexico”，防止匹到 mexico
    return any(rx.search(t) for rx in ring_regexes)

df["on_ring_of_fire"] = df["_text"].apply(on_ring_of_fire)
df[["_text","on_ring_of_fire"]].head()


,_text,on_ring_of_fire
0,"| 9km ne of aguanga, ca | california | m 1.4 ...",True
1,"| 9km ne of aguanga, ca | california | m 1.3 ...",True
2,"| 8km ne of aguanga, ca | california | m 3.4 ...",True
3,"| 9km ne of aguanga, ca | california | m 0.4 ...",True
4,"| 10km nw of avenal, ca | california | m 2.2 ...",True


In [ ]:
#e
counts = df["on_ring_of_fire"].value_counts(dropna=False)
ring_count   = int(counts.get(True, 0))
outside_count = int(counts.get(False, 0))
print("(e) Ring of Fire:", ring_count, " | Outside:", outside_count)


(e) Ring of Fire: 7189  | Outside: 2143


In [ ]:
#f
if tsunami_col is not None:
    tsunami_count_ring = int(df.loc[df["on_ring_of_fire"], tsunami_col].fillna(0).astype(int).sum())
else:
    tsunami_count_ring = np.nan

print("(f) Tsunami count along Ring of Fire:", "N/A" if pd.isna(tsunami_count_ring) else tsunami_count_ring)


(f) Tsunami count along Ring of Fire: 45
